In [46]:
import os
import json
import requests # pip install requests
import random
import time
from django.core.files.base import ContentFile
from shop.models import Category, Shop, Item
from urllib.parse import urljoin
from fake_useragent import UserAgent # pip install fake-useragent

In [47]:
class Yogiyo:
    HOST = 'https://www.yogiyo.co.kr'
    
    def __init__(self, headers):
        self.headers = headers
        
    def get_restaurant_list_by_geo(self, zip_code):
        url = self.HOST + '/api/v1/restaurants-geo/'
        params = dict(
            items=20,
            order='rank',
            page=0,
            search='',
            zip_code=zip_code,
        )
        res = requests.get(url, params=params, headers=self.headers)
        return res.json()
    
    def get_menu_list(self, restaurant_id): # restaurant_id를 지정해주면 그 레스토랑의 메뉴를 가지고 오겠다
        base_url = self.HOST + '/api/v1/restaurants/{restaurant_id}/menu/?add_photo_menu=original'
        url = base_url.format(restaurant_id=restaurant_id)
        res = requests.get(url, headers=self.headers)
        return res.json()

In [48]:
## 활용 : 식당등록

In [49]:
user_agent = UserAgent()

In [50]:
headers = {
    'X-ApiKey': 'iphoneap',
    'X-ApiSecret': 'fe5183cc3dea12bd0ce299cf110a75a2',
    'X-MOD-SBB-CTYPE': 'xhr',
    'User-Agent': user_agent.Chrome,
}

yogiyo = Yogiyo(headers)

In [51]:
zip_code = '120110'
restaurants = yogiyo.get_restaurant_list_by_geo('120110')['restaurants']
restaurants

[{'rating': 200039.783495783,
  'subtitle': '',
  'additional_discount_currently_active': False,
  'free_delivery_threshold': 0,
  'delivery_fee_explanation': '최소 주문 금액 10000원 이상',
  'is_available_delivery': True,
  'threshold': 10000,
  'adv_distance': None,
  'open': True,
  'discount_until': None,
  'central_billing': False,
  'city': '서울',
  'review_reply_count': 0,
  'sms_bonus': False,
  'franchise_id': 69,
  'phone_downlisted': False,
  'franchise_name': '(주)롯데지알에스 / 롯데리아',
  'review_image_count': 96,
  'top100_restaurant': False,
  'is_available_pickup': False,
  'additional_discount_per_menu': 0,
  'delivery_fee': 0,
  'additional_discount_pickup': 0,
  'phone': '050352594304',
  'additional_discount_only_for_user': False,
  'has_terminal': True,
  'categories': ['1인분주문', '피자양식', '치킨', '프랜차이즈', '야식'],
  'min_order_amount': 10000,
  'distance': 1.3448337975624016,
  'logo_url': '/media/restaurant_logos/롯데리아_20160408_Franchise이미지약정서_crop_200x200.jpg',
  'except_cash': True,
  'n

In [52]:
for rest in restaurants:
    category_name = rest['categories'][0]
    print(category_name)
    category = Category.objects.get(name=category_name) # 요기요는 M:N 관계 => 지정 name의 Category가 없다면, Category.DoesNotExist 예외 발생
    latlng = '{lat}, {lng}'.format(**rest)
    logo_url = urljoin(Yogiyo.HOST, rest['logo_url'])
    shop = Shop(category=category, name=rest['name'], latlng=latlng, meta=rest)
    logo_name = os.path.basename(logo_url)
    logo_data = requests.get(logo_url).content
    shop.photo.save(logo_name, ContentFile(logo_data), save=False)
    print('{name}: {lat}, {lng}, {categories}, {logo_url}'.format(**rest))
    
    shop.save()

1인분주문


DoesNotExist: Category matching query does not exist.

In [53]:
for shop in Shop.objects.all():
    print(shop.name, shop.latlng)


맥시칸치킨-연희점 37.5628544, 126.9307123
멕시카나-신촌점 37.5547722235414, 126.941644837675
원할머니보쌈-신촌점 37.5574104353704, 126.935986925105
디디치킨-연세대점 37.5661219586013, 126.931808950602
냉면앤고기어때 37.5747968607615, 126.922656131739
가화만사성24시 37.55706, 126.9361078
네네치킨-이대역점 37.5570037, 126.9414717
공룡치킨-신촌점 37.5538226251049, 126.93177389498
네네치킨-연희점 37.5659973, 126.930478
혼밥혼술-신촌점 37.5556092705005, 126.940414980044
범벅치킨-신촌점 37.5578773, 126.9346843
홍푸드스테이크시리즈 37.5526264, 126.93543
네네치킨-북가좌2점 37.580757208118, 126.915095502055
유자유김치떡볶이-신촌점 37.5590514418098, 126.935642820668
닭꾸버-서울1호신촌점 37.5573034385236, 126.932320878454
명품임실치즈피자-신촌점 37.5563502089596, 126.943191206591
노랑통닭-명지대점 37.5787759999432, 126.9239699
지브로통삼겹살전문점 37.5747968607615, 126.922656131739
굽네치킨-서대문연희점 37.5732125, 126.9353151
직화신불닭-강북 37.5817391, 126.9356768
롯데리아-건대역점 37.5407068, 127.0714389
강정구의피자생각-광진점 37.5564630424343, 127.0758847565
버거킹-건대화양점(24) 37.5409386584498, 127.068494238588
팔당왕족발 37.5620419, 127.0824028
불스바베큐-중곡점 37.5625387132301, 127.0833

In [56]:
for shop in Shop.objects.all():
    restaurant_id = shop.meta['id']
    menu_list = yogiyo.get_menu_list(restaurant_id)
    print(shop.name)
    for sub_menu_list in menu_list:
        items = sub_menu_list['items']
        for item_meta in items:
            item = Item(shop=shop, name=item_meta['name'], amount=item_meta['price'], meta=item_meta)
            
            item_image_url = item_meta.get('image', '') # iamge가 없는 메뉴가 있으면 '' 처리
            if item_image_url:
                item_image_url = urljoin(Yogiyo.HOST, item_meta['image'])
                
                item_image_name = os.path.basename(item_image_url)
                item_image_data = requests.get(item_image_url).content
                item.photo.save(item_image_name, ContentFile(item_image_data), save=False)
                
            print('saving item: {}'.format(item.name))
            item.save()

맥시칸치킨-연희점
saving item: 후라이드 치킨
saving item: 핫후라이드치킨
saving item: 양념치킨
saving item: 닭강정
saving item: 마늘간장치킨
saving item: 고추크러시
saving item: 반반치킨（후라이드반＋양념반）
saving item: 순살반반（후라이드반＋양념반）
saving item: 후라이드 치킨
saving item: 세트1（순살후라이드＋순살양념＋콜라1.25L）
saving item: 닭강정
saving item: 양념치킨
saving item: 순살후라이드
saving item: 세트2（순살후라이드＋순살마늘간장＋콜라1.25L）
saving item: 순살양념치킨 
saving item: 마늘간장치킨
saving item: 고추크러시
saving item: 순살고추크러시
saving item: 치즈랑치킨 후라이드
saving item: 치즈랑치킨 순살
saving item: 갈릭치킨
saving item: 순살갈릭
saving item: 가라아게（닭가슴살）
saving item: 춘천식 닭갈비 볶음밥
saving item: 안동식 찜닭 볶음밥
saving item: 후라이드 치킨
saving item: 핫후라이드치킨
saving item: 양념치킨
saving item: 반반치킨（후라이드반＋양념반）
saving item: 마늘간장치킨
saving item: 닭강정
saving item: 어니언치킨
saving item: 깨범벅 치킨
saving item: 칠리치킨
saving item: 핫 반반
saving item: 양념반 매운양념반
saving item: 후라이드반 치즈반
saving item: 양념반 치즈반
saving item: 후라이드반 간장반
saving item: 양념반 간장반
saving item: 후라이드반 칠리반
saving item: 후라이드반 강정반
saving item: 양념반 양썩반
saving item: 후라이드반 깨범벅반
saving item: 칠리반 간장반
sa

saving item: 볶음밥
saving item: 탕수육 小
saving item: 송이덮밥
saving item: 볶음밥
saving item: 삼선볶음밥
saving item: 새우볶음밥
saving item: XO볶음밥
saving item: XO게살볶음밥
saving item: 게살볶음밥
saving item: 육개장
saving item: 짬뽕밥
saving item: 삼선짬뽕밥
saving item: 잡채밥
saving item: 특밥
saving item: 마파두부밥
saving item: 송이덮밥
saving item: 고추잡채밥
saving item: 유산슬밥
saving item: 잡탕밥
saving item: 제육덮밥
saving item: 오므라이스
saving item: 짜장면
saving item: 간짜장
saving item: 삼선쟁반짜장 2人
saving item: 삼선간짜장
saving item: 짬뽕
saving item: 삼선볶음짬뽕（2人）
saving item: 삼선짬뽕
saving item: 삼선고추짬뽕
saving item: 울면
saving item: 우동
saving item: 삼선우동
saving item: 사천탕면
saving item: 청양고추볶음짜장
saving item: 청양고추볶음짬뽕
saving item: 굴짬뽕
saving item: 치즈짬뽕
saving item: 황제짬뽕
saving item: 기스면
saving item: 물만두
saving item: 군만두
saving item: 꽃빵
saving item: 짬짜면
saving item: 볶짬면
saving item: 볶짜면
saving item: 탕짜면
saving item: 탕짬면
saving item: 탕볶밥
saving item: 탕수육 小
saving item: 탕수육 中
saving item: 탕수육 大
saving item: 사천탕수육 小
saving item: 사천탕수육 中
saving item: 사천탕수육 大
saving ite

saving item: 즉석밥
saving item: 캔 콜라
saving item: 콜라 500mL
saving item: 사이다 500mL
saving item: 콜라 1.25L
saving item: 사이다 1.25L
홍푸드스테이크시리즈
saving item: 해물떡볶이
saving item: 오뎅탕
saving item: 만두국
saving item: 떡만두국
saving item: 제육볶음
saving item: 낙지볶음
saving item: 삼겹살구이＋미니된장 도시락
saving item: 콜라1.25L
saving item: 스팸김치찌개
saving item: 사이다1.5L
saving item: 수제등심통돈까스
saving item: 오뎅김치찌개
saving item: 웰빙스팸도시락
saving item: 돼지불백 도시락
saving item: 제육볶음
saving item: 생선구이＋미니된장（고등어＋삼치）도시락
saving item: 특제육덮밥
saving item: 칼칼한순두부
saving item: 참치김치볶음밥
saving item: 돼지고기묵은지찜
saving item: 장조림버터비빔밥
saving item: 김치제육치즈밥
saving item: 새우오무라이스
saving item: 삼겹살백반＋미니된장
saving item: 제육볶음백반＋미니된장
saving item: 홍스테이크 미디움 도시락
saving item: 삼겹살혼족 세트（1인）
saving item: 얇게구운 삼겹살 고추장볶음 세트
saving item: 얇게 구운 삼겹살세트
saving item: 제육 볶음 고기 세트
saving item: 오징어 제육볶음（오삼） 세트
saving item: 돼지숯불고기세트
saving item: 목살김치찌개
saving item: 칼칼한순두부찌개
saving item: 순한순두부찌개
saving item: 치즈순두부
saving item: 햄순두부
saving item: 소고기순두부
saving item: 버섯순두부
saving item

saving item: 허니버터 포테이토 피자 L
saving item: 더블치킨찹스테이크 피자 L
saving item: 베이컨체다프라이 피자 L
saving item: 불닭 피자 L
saving item: 쉬림프골드 피자 M
saving item: 베이컨포테이토골드 피자 M
saving item: 파이바이트 피자 M
saving item: 직화스테이크 피자 M
saving item: 쉬림프골드 피자 L
saving item: 베이컨포테이토골드 피자 L
saving item: 파이바이트 피자 L
saving item: 직화스테이크 피자 L
saving item: 대박세트（오리지널크러스트 L＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET1（명품오리지널피자 L＋치즈스틱 5개＋버팔로윙 5개＋콜라 1.25L）
saving item: SET2（명품오리지널피자 L＋버팔로윙 5개 or 치즈스틱 5개＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET3 （명품오리지널피자 L＋치킨텐더 6개＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET4（명품프리미엄피자L＋버팔로윙 5개＋치즈스틱 5개＋콜라 1.25L）
saving item: SET5（명품프리미엄피자L＋버팔로윙 5개 or 치즈스틱 5개＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET6（명품프리미엄피자L＋치즈오븐스파게티＋치킨텐더 6개＋콜라 1.25L）
saving item: SET7（명품오리지널크러스트L＋치즈스틱 5개＋버팔로윙 5개＋콜라 1.25L）
saving item: SET8（명품오리지널크러스트L＋버팔로윙 5개 or 치즈스틱 5개＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET9（명품오리지널크러스트L＋치킨텐더 6개＋치즈오븐스파게티＋콜라 1.25L）
saving item: SET10（명품골드피자L＋버팔로윙 5개＋치즈스틱 5개＋콜라 1.25L）
saving item: SET11（명품골드피자L＋버팔로윙 5개 or 치즈스틱 5개＋콜라 1.25L）
sa

saving item: 모짜렐라인더 버거-베이컨 세트
saving item: 모짜렐라인더 버거-해쉬 세트
saving item: 모짜렐라인더 버거-더블 세트
saving item: 와규오리지널세트
saving item: 데리세트
saving item: 불고기버거세트
saving item: 새우버거세트
saving item: 원조빅불세트
saving item: 핫크리스피버거세트
saving item: 클래식치즈버거 세트
saving item: 해쉬브라운
saving item: 양념감자
saving item: 오징어링
saving item: 치즈스틱
saving item: 베이컨롱치즈스틱
saving item: 롱치즈스틱
saving item: 콘샐러드
saving item: 포테이토
saving item: 오렌지（PET-400ml）
saving item: 레몬에이드
saving item: 사이다（R）
saving item: 아메리카노
saving item: 아이스 아메리카노
saving item: 아이스 아메리카노（L）
saving item: 아이스 카페라떼
saving item: 카페라떼
saving item: 콜라（R）
saving item: 콜라1.25L（PET）
saving item: 사이다1.25L（PET）
saving item: 망고피치
saving item: 아이스초코
saving item: 블루소다
saving item: 화이어윙 2조각
saving item: 화이어윙 4조각
saving item: 순살치킨 풀팩
saving item: 순살치킨 하프팩
saving item: 치킨 1조각
saving item: 치킨패밀리팩-콘샐러드
saving item: 치킨풀팩-콘샐러드
saving item: 치킨하프팩-콘샐러드
saving item: 치킨패밀리팩-포테이토
saving item: 치킨풀팩-포테이토
saving item: 치킨하프팩-포테이토
saving item: 치킨휠레 2조각
saving item: 치킨휠레 4조각
saving item: 모짜렐라

...

saving item: 보쌈 中（2~3人）국내산 목살, 삼겹살
saving item: 보쌈 大（3~4人）국내산 목살, 삼겹살
saving item: 보쌈 특大（4~5人）국내산 목살, 삼겹살
saving item: 보족 小
saving item: 보족 中
saving item: 보족 大
saving item: 보족 특大
saving item: 쟁반국수 1人
saving item: 부침개
saving item: 보속김치（무）
saving item: 야채
saving item: 공기밥
saving item: 콜라 1.25L
saving item: 스프라이트 1.5L
불스바베큐-중곡점
saving item: 불스숯불바베큐
saving item: 불스숯불양념바베큐
saving item: 불스허브후라이드（뼈）
saving item: 불스순살후라이드
saving item: 불스순살후라이드양념
saving item: 불스버팔로윙（16조각）
saving item: 불스 감자 튀김 R
saving item: 불스숯불양념바베큐
saving item: 불스숯불바베큐（반）＋불스숯불양념（반）
saving item: 새우튀김 10조각
saving item: 고구마 튀김 R
saving item: 불스 감자 튀김 L
saving item: 불스숯불바베큐
saving item: 불스숯불양념（반）＋불스숯불간장（반）
saving item: 치즈 추가
saving item: 안심탕수육 小
saving item: 불스숯불바베큐
saving item: 불스숯불양념바베큐
saving item: 불스숯불간장바베큐
saving item: 불스숯불바베큐（반）＋불스숯불양념（반）
saving item: 불스숯불바베큐（반）＋불스숯불간장（반）
saving item: 불스숯불양념（반）＋불스숯불간장（반）
saving item: 불스오동통순살바베큐
saving item: 불스오동통순살양념바베큐
saving item: 불스오동통순살간장바베큐
saving item: 불스순살치즈그라탕
saving item: 불스허브후라이드

saving item: 간장뀌바로 中
saving item: 간장뀌바로 大
saving item: 양념감자튀김（R）
saving item: 양념감자튀김（L）
saving item: 조각고구마튀김（R）
saving item: 조각고구마튀김（L）
saving item: 닭날개봉튀김（8조각）
saving item: 닭날개봉튀김（16조각）
saving item: 생새우튀김 10조각
saving item: 생새우튀김 5조각	
saving item: 코카콜라 500mL
saving item: 코카콜라 1.25L
saving item: 스프라이트 500mL
saving item: 스프라이트 1.5L
천호동똥찌비
saving item: 똥집 통마늘 구이
saving item: 뼈없는 닭발
saving item: 똥반오반 통마늘 구이
saving item: 똥반닭반 통마늘 구이
saving item: 오반닭반 통마늘 구이
saving item: 오징어 통마늘 구이
saving item: 똥집 통마늘 구이
saving item: 날치알주먹밥
saving item: 돼지껍데기（1인분）
saving item: 똥반오반 통마늘 구이
saving item: 모듬 통마늘 구이
saving item: 치즈 추가
saving item: 계란찜
saving item: 치즈 계란말이
saving item: 그냥 계란말이
saving item: 뼈없는 닭발
saving item: 똥집 통마늘 구이
saving item: 똥반오반 통마늘 구이
saving item: 똥반닭반 통마늘 구이
saving item: 오반닭반 통마늘 구이
saving item: 오징어 통마늘 구이
saving item: 똥반돈반 통마늘 구이
saving item: 모듬 통마늘 구이
saving item: 뼈없는 닭발
saving item: 뼈있는 닭발
saving item: 마늘 추가
saving item: 떡 추가
saving item: 치즈 추가
saving item: 그냥 계란말이
saving item: 치즈 계란말

saving item: 치킨하프팩-콘샐러드
saving item: 치킨패밀리팩-포테이토
saving item: 치킨풀팩-포테이토
saving item: 치킨하프팩-포테이토
saving item: 치킨휠레 2조각
saving item: 치킨휠레 4조각
saving item: 모짜렐라더블팩
saving item: 홈더블팩
saving item: 홈패밀리팩
saving item: 데리야끼소스
saving item: 머스타드소스
saving item: 핫소스
saving item: 휠레소스_허니머스타드
saving item: 휠레소스_스위트앤샤워
saving item: 시즈닝_치즈맛
saving item: 시즈닝_칠리맛
saving item: 시즈닝_오니언맛
강정구의피자생각-광진점
saving item: 크림치즈포크피자 L
saving item: 크림치즈포크피자 L
saving item: 베이컨 시카고피자
saving item: 불닭 시카고피자
saving item: 베이컨칩스피자 L 
saving item: 포테이토골드피자 L
saving item: 치즈크러스트세트1（치즈크러스트피자L 1판＋치즈오븐스파게티＋콜라 1.25L）
saving item: 오리지널세트1（오리지널피자L 1판＋치즈오븐스파게티＋콜라 1.25L）
saving item: 베이컨 시카고피자
saving item: 마약스파게티
saving item: 치즈오븐스파게티
saving item: 버팔로 윙（10개）
saving item: 고구마리치골드피자 L
saving item: 콤비네이션피자 L
saving item: 콤비네이션피자 M
saving item: 까르보나라 
saving item: 크림치즈포크피자 M
saving item: 크림치즈포크피자 L
saving item: 크랜베리 시카고피자
saving item: 베이컨 시카고피자
saving item: 불닭 시카고피자
saving item: 페페로니 시카고피자
saving item: 포테이토골드피자 L
saving item: 고구마리치골드피자 L
s

saving item: 닭윙（날개부위）
saving item: 닭봉（어깨부위）
saving item: 오돌뼈
saving item: 마늘소금똥집
saving item: 마늘양념똥집
saving item: 양념구이불꼼장어
saving item: 양념구이불족발
saving item: 불막창숯불구이
saving item: 국물떡볶이
saving item: 주먹밥
saving item: 스팸주먹밥
saving item: 김치주먹밥
saving item: 날치알주먹밥
saving item: 참치주먹밥
saving item: 멸치주먹밥
saving item: 오돌뼈주먹밥
saving item: 계란찜
saving item: 누룽지
saving item: 콜라500mL
saving item: 콜라1.5L
saving item: 쿨피스930mL
saving item: 생수
평양불냉면&시골고기냉면
saving item: 물냉면
saving item: 비빔냉면
saving item: 열무（물）냉면
saving item: 회（홍어）비빔냉면
saving item: 족발 200g 추가
saving item: 보쌈 200g 추가
saving item: 물냉면
saving item: 비빔냉면
saving item: 사계절 왕만두세트（왕만두4개＋갈비만두6개）
saving item: 갈비만두 6개
saving item: 명품 보쌈 비빔냉면
saving item: 냉면사리 추가
saving item: 열무（물）냉면
saving item: 명품 족발 비빔냉면
saving item: 불 비빔냉면
saving item: 냉면육수 추가
saving item: 물냉면
saving item: 비빔냉면
saving item: 불 물냉면
saving item: 불 비빔냉면
saving item: 열무（물）냉면
saving item: 명품 족발 비빔냉면
saving item: 명품 보쌈 비빔냉면
saving item: 회（홍어）비빔냉면
saving item: 사계절 왕만두세트（왕만두4개＋갈비만두6개）
sav

saving item: 옛날 순살오븐바베큐
saving item: 옛날 순살양념
saving item: 핫순살 그라탕
saving item: 옛날 순살오븐바베큐（반）＋옛날 순살양념（반）
saving item: 옛날감자튀김 R
saving item: 옛날감자튀김 L
saving item: 고구마튀김 R
saving item: 고구마튀김 L
saving item: 옛날치즈스틱 5개
saving item: 옛날치즈스틱 12개
saving item: 옛날새우튀김 10조각
saving item: 옛날새우튀김 5조각
saving item: 코카콜라 500mL
saving item: 코카콜라 1.25L
saving item: 스프라이트 500mL
saving item: 스프라이트 1.5L
가장맛있는족발-장안사거리점
saving item: 족발
saving item: 냉채족발 大
saving item: 보쌈
saving item: 반반족발（족발＋불족）
saving item: 큰쟁반국수
saving item: 직화 불족발
saving item: 반반족발（족발＋불족）
saving item: 족발
saving item: 보쌈
saving item: 가족의 족발보쌈
saving item: 직화 불족발
saving item: 큰쟁반국수
saving item: 냉채족발 大
saving item: 해물부추전
saving item: 얼큰술국
saving item: 바베큐 마늘족발
saving item: 족발
saving item: 흑 마늘족발
saving item: 허니 마늘족발
saving item: 바베큐 마늘족발
saving item: 직화 불족발
saving item: 보쌈
saving item: 냉채족발 大
saving item: 가족의 족발보쌈
saving item: 반반족발（족발＋불족）
saving item: 큰쟁반국수
saving item: 해물부추전
saving item: 가마보꼬 모듬오뎅탕
saving item: 얼큰술국
saving item: 가족의 불족
saving 